# Neo4J and LLM setup 

### Import Libraries

In [ ]:
from langchain_community.graphs import Neo4jGraph
from langchain_openai import AzureChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

### Create a Neo4J Graph Object

In [ ]:
graph = Neo4jGraph(url="***********",username="**************",password="************")

### Create a LLM Model

In [ ]:
llm = AzureChatOpenAI(temperature=0,
                     model="gpt-4o",
                     deployment_name="*************",
                     api_key="*********************",
                     api_version="*****************",
                     azure_endpoint="**************",
                     openai_api_type="azure")

### Pass a graph to llm

In [ ]:
CYPHER_GENERATION_TEMPLATE = """ Task:Generate cypher statement to query Graph databse.
Instructions:
1. Use only provided relationship types and properties in Schema.
2. Retain the direction of relationship.

Schama: {schema}
Question : {question}

"""

CYPHER_GENERATION_PROMPT = PromptTemplate(input_variables=["schema","qustion"],template=CYPHER_GENERATION_TEMPLATE)

In [ ]:
QA_GENERATION_TEMPLATE = """ You are an assistant that takes the result from a Neo4J Cypher Query and forms a human-readable response.

Query Results : {context}

Question : {question}

"""

CYPHER_GENERATION_PROMPT = PromptTemplate(input_variables=["qustion"],template=QA_GENERATION_TEMPLATE)

In [ ]:
chain = GraphCypherQAChain.from_llm(llm,
                                   graph=graph,
                                   verbose=True,
                                   top_k=50,
                                   cypher_prompt = CYPHER_GENERATION_PROMPT,
                                   qa_prompt = QA_GENERATION_PROMPT,
                                   return_intermediate_steps=True)

### Run a query 

In [ ]:
query = "Query"
output = chain(query)
print("Answer : ", output['result'])
print("Cypher Query : ",output['intermediate_steps'][0])